#**Load in the data**




We combine the fake and true data, a veritable jumble!
Our data is sourced from the University of Victoria, in British Columbia, Canada. 





In [16]:
import os

In [17]:
os.listdir()

['.config', 'train.tsv', 'Fake.csv', 'True.csv', 'valid.tsv', 'sample_data']

In [18]:
import pandas as pd
import numpy as np
import re, string
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import Sequential
from keras.preprocessing import text, sequence
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import pandas as pd

data = pd.read_csv("train.tsv", sep="\t")

data.reset_index(inplace = True)

data.drop('index', axis = 1, inplace = True)

valdata = pd.read_csv("valid.tsv", sep="\t")

valdata.reset_index(inplace = True)

valdata.drop('index', axis = 1, inplace = True)



# **Get Stopwords**
Use nltk to get stopwords -- words that are very common

In [19]:
import nltk
import string

nltk.download('stopwords')

stopword = nltk.corpus.stopwords.words('english')
#punctuations = list(string.punctuation)
#stopword = stopword + punctuations
#print(stopword)
data

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,file,valid,text,topic,person,position,state,party,a,b,c,d,e,context
0,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
1,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
2,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
3,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
4,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4441,5479.json,false,When (Marcy) Kaptur fought for the Democrats p...,"health-care,message-machine-2012,voting-record",marcy-kaptur,Representative from Ohio's Ninth Congressional...,Ohio,democrat,0.0,2.0,4.0,4.0,0.0,a campaign commercial
4442,13021.json,false,I released the most extensive financial review...,"candidates-biography,taxes,transparency",donald-trump,President-Elect,New York,republican,63.0,114.0,51.0,37.0,61.0,an interview with ABC News
4443,6527.json,half-true,The Obama administration willfully leaked the ...,"foreign-policy,message-machine-2012,military,n...",special-operations-opsec-education-fund,NaN,NaN,none,1.0,1.0,1.0,0.0,0.0,a Web video
4444,7777.json,barely-true,When Susan Rice spoke about Benghazi on Sunday...,"foreign-policy,terrorism",jay-carney,White House press secretary,District of Columbia,none,2.0,2.0,1.0,2.0,0.0,a White House press briefing


#**Cleaning the data**

Preparing the data before we make it readable to machine

In [20]:
from pandas.io.parsers.readers import TextFileReader
from numpy import NaN
import re

def removestopwords(txt):
  ans = []
  txt = txt[0]
  txt = re.sub(r'\([^]]*\)', ' ', txt) # removing parenthesis
  txt = re.sub(r'\[[^]]*\]', ' ', txt) # removing third brackets
  txt = re.sub(r'([\.\\\+\*\?\[\^\]\$\(\)\{\}\!\<\>\|\:\-\,\"\”\“\‘])', ' ', txt)
 # print(txt[0].split())
  for i in txt.split():
    if (i.lower().strip()) not in stopword:
      ans.append((i.lower().strip()))
  #print(ans)
  ans.append(" ")
  return " ".join(ans)

#this word
data = data.dropna()

valdata = valdata.dropna()


data["position"] = data["position"].to_frame(0).apply(removestopwords, axis = 1)
data["text"] = data["text"].to_frame(0).apply(removestopwords, axis = 1)
data["state"] = data["state"].to_frame(0).apply(removestopwords, axis = 1)
data["person"] = data["person"].to_frame(0).apply(removestopwords, axis = 1)
data["context"] = data["context"].to_frame(0).apply(removestopwords, axis = 1)
data["topic"] = data["topic"].to_frame(0).apply(removestopwords, axis = 1)


valdata["position"] = valdata["position"].to_frame(0).apply(removestopwords, axis = 1)
valdata["text"] = valdata["text"].to_frame(0).apply(removestopwords, axis = 1)
valdata["person"] = valdata["person"].to_frame(0).apply(removestopwords, axis = 1)
valdata["state"] = valdata["state"].to_frame(0).apply(removestopwords, axis = 1)
valdata["context"] = valdata["context"].to_frame(0).apply(removestopwords, axis = 1)
valdata["topic"] = valdata["topic"].to_frame(0).apply(removestopwords, axis = 1)


data




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,file,valid,text,topic,person,position,state,party,a,b,c,d,e,context
0,10540.json,half-true,decline coal start started natural gas took st...,energy history job accomplishments,scott surovell,state delegate,virginia,democrat,0.0,0.0,1.0,1.0,0.0,floor speech
1,324.json,mostly-true,hillary clinton agrees john mccain voting give...,foreign policy,barack obama,president,illinois,democrat,70.0,71.0,160.0,163.0,9.0,denver
4,12465.json,true,chicago bears starting quarterbacks last 10 ye...,education,robin vos,wisconsin assembly speaker,wisconsin,republican,0.0,3.0,2.0,5.0,1.0,online opinion piece
6,153.json,half-true,i'm person stage worked actively last year pas...,ethics,barack obama,president,illinois,democrat,70.0,71.0,160.0,163.0,9.0,democratic debate philadelphia pa
8,9741.json,mostly-true,says gop primary opponents glenn grothman joe ...,energy message machine 2014 voting record,duey stroebel,state representative,wisconsin,republican,0.0,0.0,0.0,1.0,0.0,online video
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4440,10405.json,false,president barack obamas policies resulted gas ...,energy job accomplishments,bobby scott,u congressman,virginia,democrat,1.0,2.0,1.0,5.0,0.0,news release
4441,5479.json,false,kucinich voted,health care message machine 2012 voting record,marcy kaptur,representative ohio's ninth congressional dist...,ohio,democrat,0.0,2.0,4.0,4.0,0.0,campaign commercial
4442,13021.json,false,released extensive financial review anybody hi...,candidates biography taxes transparency,donald trump,president elect,new york,republican,63.0,114.0,51.0,37.0,61.0,interview abc news
4444,7777.json,barely-true,susan rice spoke benghazi sunday news shows sa...,foreign policy terrorism,jay carney,white house press secretary,district columbia,none,2.0,2.0,1.0,2.0,0.0,white house press briefing


In [21]:
data["text"] = data["topic"] + data["person"] + data["position"] + data["state"] + data["party"] + data["text"]

valdata["text"] = valdata["topic"] + valdata["person"] + valdata["position"] + valdata["state"] + valdata["party"] + valdata["text"]

#I combined training and validation here, I split them up later
#using train_test_split function
data = pd.concat([valdata, data])
convertdict = {"pants-fire": 0, "false": 0, "barely-true": 0, "half-true": 1, "mostly-true": 1, "true":1}

def convert(txt):
  return convertdict[txt[0]]

data["valid"] = data["valid"].to_frame(0).apply(convert, axis = 1)

data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,file,valid,text,topic,person,position,state,party,a,b,c,d,e,context
1,7891.json,0,campaign finance congress taxes earl blumenau...,campaign finance congress taxes,earl blumenauer,u representative,oregon,democrat,0.0,1.0,1.0,1.0,0.0,u ways means hearing
2,8169.json,1,poverty jim francesconi member state board h...,poverty,jim francesconi,member state board higher education,oregon,none,0.0,1.0,1.0,1.0,0.0,opinion article
3,929.json,1,economy stimulus barack obama president ill...,economy stimulus,barack obama,president,illinois,democrat,70.0,71.0,160.0,163.0,9.0,interview cbs news
4,9416.json,0,guns jim rubens small business owner new ha...,guns,jim rubens,small business owner,new hampshire,republican,1.0,1.0,0.0,1.0,0.0,interview gun shop hudson n h
5,6861.json,1,education state budget andy berke lawyer sta...,education state budget,andy berke,lawyer state senator,tennessee,democrat,0.0,0.0,0.0,0.0,0.0,letter state senate education committee chairw...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4440,10405.json,0,energy job accomplishments bobby scott u con...,energy job accomplishments,bobby scott,u congressman,virginia,democrat,1.0,2.0,1.0,5.0,0.0,news release
4441,5479.json,0,health care message machine 2012 voting record...,health care message machine 2012 voting record,marcy kaptur,representative ohio's ninth congressional dist...,ohio,democrat,0.0,2.0,4.0,4.0,0.0,campaign commercial
4442,13021.json,0,candidates biography taxes transparency donal...,candidates biography taxes transparency,donald trump,president elect,new york,republican,63.0,114.0,51.0,37.0,61.0,interview abc news
4444,7777.json,0,foreign policy terrorism jay carney white ho...,foreign policy terrorism,jay carney,white house press secretary,district columbia,none,2.0,2.0,1.0,2.0,0.0,white house press briefing


[link text](https://)

In [22]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data["text"], data['valid'], random_state = 0 )

In [23]:
print((y_train))

2464    0
180     1
1215    0
1288    0
1165    0
       ..
1247    1
3603    0
1216    1
2661    1
2845    1
Name: valid, Length: 2853, dtype: int64


In [24]:

from keras.preprocessing import text, sequence


tokenizer = text.Tokenizer(num_words = 10000)
tokenizer.fit_on_texts(x_train)
tokenized_train = tokenizer.texts_to_sequences(x_train)
x_train_sequence = sequence.pad_sequences(tokenized_train, maxlen = 1000)

tokenized_test = tokenizer.texts_to_sequences(x_test)
x_test_sequence = sequence.pad_sequences(tokenized_test, maxlen = 1000)



In [25]:
from keras import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.callbacks import ReduceLROnPlateau

batch_size = 256
embed_size = 200
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.5, min_lr=0.00001)




In [26]:
#embedding_matrix.shape, embed_size
x_train_sequence = x_train_sequence[:10000]

In [27]:
import tensorflow as tf 
from tensorflow import keras


model = tf.keras.models.Sequential()

model.add(Embedding(10000, output_dim = embed_size, input_length=1000))
model.add(LSTM(units = 128, activation = 'relu', recurrent_activation = 'tanh', return_sequences = True, recurrent_dropout = 0, dropout = 0.25 ))
model.add(LSTM(units = 64,activation = 'relu', recurrent_activation = 'tanh', recurrent_dropout = 0, dropout = 0.1))
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer = keras.optimizers.Adam(learning_rate = 0.01), loss = tf.keras.losses.binary_crossentropy, metrics = ['accuracy'])



In [28]:
x_train_sequence

array([[   0,    0,    0, ..., 1682,  539, 1397],
       [   0,    0,    0, ...,   16,   15,  145],
       [   0,    0,    0, ...,  706, 3967,   11],
       ...,
       [   0,    0,    0, ..., 8546,  428,  880],
       [   0,    0,    0, ..., 1594,  553,  576],
       [   0,    0,    0, ...,   22,  328,  362]], dtype=int32)

In [29]:
x_train_sequence.shape, y_train.shape

((2853, 1000), (2853,))

In [30]:
history = model.fit(x_train_sequence, y_train, batch_size = batch_size , validation_split = 0.1 , epochs = 15)



Epoch 1/15
11/11 [==============================] - 56s 5s/step - loss: 0.6836 - accuracy: 0.5621 - val_loss: 0.6988 - val_accuracy: 0.5315
Epoch 2/15
11/11 [==============================] - 43s 4s/step - loss: 0.6816 - accuracy: 0.5769 - val_loss: 0.6955 - val_accuracy: 0.5315
Epoch 3/15
11/11 [==============================] - 43s 4s/step - loss: 0.6813 - accuracy: 0.5769 - val_loss: 0.6943 - val_accuracy: 0.5315
Epoch 4/15
11/11 [==============================] - 43s 4s/step - loss: 0.6814 - accuracy: 0.5769 - val_loss: 0.6959 - val_accuracy: 0.5315
Epoch 5/15
11/11 [==============================] - 43s 4s/step - loss: 0.6817 - accuracy: 0.5769 - val_loss: 0.6969 - val_accuracy: 0.5315
Epoch 6/15
11/11 [==============================] - 43s 4s/step - loss: 0.6814 - accuracy: 0.5769 - val_loss: 0.6958 - val_accuracy: 0.5315
Epoch 7/15
11/11 [==============================] - 43s 4s/step - loss: 0.6815 - accuracy: 0.5769 - val_loss: 0.6941 - val_accuracy: 0.5315
Epoch 8/15
11/11 [==

In [31]:
print(model.evaluate(x_test_sequence,y_test))

30/30 [==============================] - 8s 255ms/step - loss: 0.6809 - accuracy: 0.5788
[0.6808534860610962, 0.5787814855575562]
